<a href="https://colab.research.google.com/github/hagarbarakat/bigint-prime-check/blob/main/Challenge1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
  !git clone https://github.com/google/benchmark.git
!git clone https://github.com/google/googletest.git benchmark/googletest

Cloning into 'benchmark'...
remote: Enumerating objects: 9891, done.
remote: Counting objects: 100% (217/217), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 9891 (delta 164), reused 106 (delta 106), pack-reused 9674 (from 3)
Receiving objects: 100% (9891/9891), 3.19 MiB | 10.67 MiB/s, done.
Resolving deltas: 100% (6635/6635), done.
Cloning into 'benchmark/googletest'...
remote: Enumerating objects: 28346, done.
remote: Counting objects: 100% (158/158), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 28346 (delta 88), reused 47 (delta 42), pack-reused 28188 (from 3)
Receiving objects: 100% (28346/28346), 13.56 MiB | 11.75 MiB/s, done.
Resolving deltas: 100% (21023/21023), done.


In [ ]:
!rm -rf benchmark/build
!cmake -E make_directory "benchmark/build"
!cmake -E chdir "benchmark/build" cmake -DCMAKE_BUILD_TYPE=Release ..
!cmake --build "benchmark/build" --config Release --target install

-- The CXX compiler identification is GNU 11.4.0
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Failed to find LLVM FileCheck
-- Found Git: /usr/bin/git (found version "2.34.1")
-- Google Benchmark version: v1.9.4-60-gd029e781, normalized to 1.9.4.60
-- Looking for shm_open in rt
-- Looking for shm_open in rt - found
-- Performing Test HAVE_CXX_FLAG_WALL
-- Performing Test HAVE_CXX_FLAG_WALL - Success
-- Performing Test HAVE_CXX_FLAG_WEXTRA
-- Performing Test HAVE_CXX_FLAG_WEXTRA - Success
-- Performing Test HAVE_CXX_FLAG_WSHADOW
-- Performing Test HAVE_CXX_FLAG_WSHADOW - Success
-- Performing Test HAVE_CXX_FLAG_WFLOAT_EQUAL
-- Performing Test HAVE_CXX_FLAG_WFLOAT_EQUAL - Success
-- Performing Test HAVE_CXX_FLAG_WOLD_STYLE_CAST
-- Performing Test HAVE_CXX_FLAG_WOLD_STYLE_CAST - Success
-- Performing Test HAVE_CXX_FLAG_WCON

In [ ]:
%%writefile power_computation.cpp
#include <boost/multiprecision/cpp_int.hpp>
#include <benchmark/benchmark.h>
#include <iostream>
using namespace boost::multiprecision;
using namespace boost::multiprecision::literals;
using uint512 = number<cpp_int_backend<512, 512, unsigned_magnitude, unchecked, void>>;
using uint1024 = number<cpp_int_backend<1024, 1024, unsigned_magnitude, unchecked, void>>;


uint1024 bpow(uint512  a,uint512 n)
{   uint1024 half;
    if (n==0)
        return 1;
    if (n%2 == 0)
    {
        half=bpow(a,n/2);
        return half*half;
    }
    else
    {
        half=bpow(a,(n-1)/2);
        return half*half*a;
    }
}

uint1024 powmod(uint512  a,uint512 p,uint512 n)
{
  uint1024 x = bpow(a,p);
  return x % n;
}
uint1024 primTest(uint512 n)
{
  uint512 a = 2 + rand() % (n - 3);
  uint1024 result = powmod(a,n-1,n);
  if (result != 1)
      return false;
  else
      return true;
}
// static void BM_PrimTest(benchmark::State& state)
// {
//     uint512  n=state.range(0);
//     std::cout<<n<<std::endl;
//     for (auto _ :state)
//         primTest(n);
//     state.SetComplexityN(state.range(0));
// }


// BENCHMARK(BM_PrimTest)
//     ->RangeMultiplier(2)
//     ->Range(1<<9, 1<<20)
//     ->Complexity();

static void BM_PrimalityTestRange(benchmark::State& state) {
    int bits = state.range(0);
    uint512 n = (uint512(1) << bits) - 1;
    std::cout<<n<<std::endl;

    for (auto _ : state) {
        uint1024 result = primTest(n);
        benchmark::DoNotOptimize(result);
    }

    state.SetComplexityN(state.range(0));
}

// Tests
void printTests() {
   uint1024 test_2 = primTest(2);
  std::cout<<test_2<<std::endl;

  uint1024 test_5 = primTest(5);
  std::cout<<test_5<<std::endl;

  uint1024 test_8 = primTest(8);
  std::cout<<test_8<<std::endl;

  uint1024 test_13 = primTest(13);
  std::cout<<test_13<<std::endl;

  uint1024 test_29 = primTest(29);
  std::cout<<test_29<<std::endl;

  uint1024 test_219 = primTest(219);
  std::cout<<test_219<<std::endl;

}


BENCHMARK(BM_PrimalityTestRange)
    ->RangeMultiplier(2)
    ->Range(32, 512)  // Test 32, 64, 128, 256, 512 bits
    ->Complexity()
    ->Unit(benchmark::kMicrosecond);

//BENCHMARK_MAIN();


int main(int argc, char** argv) {
    printTests();
     ::benchmark::Initialize(&argc, argv);
    if (::benchmark::ReportUnrecognizedArguments(argc, argv)) return 1;
    ::benchmark::RunSpecifiedBenchmarks();
    return 0;
}

Writing power_computation.cpp


In [ ]:
!g++ power_computation.cpp -O2 -std=c++11 -isystem benchmark/include -Lbenchmark/build/src -lbenchmark -lpthread -o testprogram2

In [ ]:
!./testprogram2

1
1
0
1
1
0
2025-10-18T13:47:40+00:00
Running ./testprogram2
Run on (2 X 2200 MHz CPU s)
CPU Caches:
  L1 Data 32 KiB (x1)
  L1 Instruction 32 KiB (x1)
  L2 Unified 256 KiB (x1)
  L3 Unified 56320 KiB (x1)
Load Average: 0.48, 0.40, 0.39
***WARNING*** ASLR is enabled, the results may have unreproducible noise in them.
4294967295
4294967295
4294967295
4294967295
4294967295
4294967295
--------------------------------------------------------------------
Benchmark                          Time             CPU   Iterations
--------------------------------------------------------------------
BM_PrimalityTestRange/32        9.54 us         9.53 us        65497
18446744073709551615
18446744073709551615
18446744073709551615
18446744073709551615
18446744073709551615
18446744073709551615
BM_PrimalityTestRange/64        19.1 us         19.0 us        37823
340282366920938463463374607431768211455
340282366920938463463374607431768211455
340282366920938463463374607431768211455
340282366920938463463374